In this notebook, I will try to build some Autoencoder models for anomaly detection. 
My autoencoder models are mainly based on this article 
https://towardsdatascience.com/build-the-right-autoencoder-tune-and-optimize-using-pca-principles-part-ii-24b9cca69bd6

Also, every training data is pre-processed. (I refered following notebook and pick up top-150 features)
https://www.kaggle.com/artgor/eda-and-models

I built 5 models and the last one is a conbination of others.

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import Model, load_model
from keras.layers import Input, Dense
#from keras.callbacks import ModelCheckpoint, EarlyStoping
from keras import regularizers

folder_pathin = '../input/fraud-detection-processed-dara/'
X_train = pd.read_csv(f'{folder_pathin}X_ae_150_train.csv')
X_valid = pd.read_csv(f'{folder_pathin}X_ae_150_test.csv')
y_valid = pd.read_csv(f'{folder_pathin}y_ae_test.csv')

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_valid = pd.DataFrame(scaler.transform(X_valid), columns=X_valid.columns)

In [ ]:
train_data, valid_data = train_test_split(X_train, test_size=0.2, random_state=42)

input_dim = train_data.shape[1]

In [ ]:
train_data.shape

In [ ]:
nb_epoch = 200
batch_size = 2048

#encoding_dim = int(input_dim/2)
encoding_dim = input_dim
learning_rate = 1e-3

In [ ]:
from keras.layers import Dense, BatchNormalization, Activation
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, Dense, Layer, InputSpec
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers, activations, initializers, constraints, Sequential
from keras import backend as K
from keras.constraints import UnitNorm, Constraint

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

DO_TRANING = True
 
#autoencoder.compile(optimizer='adam', 
#                    loss='mean_squared_error', 
#                    metrics=['accuracy'])
 
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)

earlystoping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')

In [ ]:
def train_AE(autoencoder):
  if DO_TRANING:
     history = autoencoder.fit(train_data, train_data,
                               epochs=nb_epoch,
                               batch_size=batch_size,
                               shuffle=True,
                               validation_data=(valid_data, valid_data),
                               verbose=1,
                               callbacks=[checkpointer, earlystoping]).history

     # Model loss
     plt.plot(history['loss'])
     plt.plot(history['val_loss'])
     plt.title('model loss')
     plt.ylabel('acc')
     plt.xlabel('epoch')
     plt.legend(['train', 'val'], loc='upper right');
     plt.show()
     
  return(autoencoder)

In [ ]:
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support)

def result_visualization(autoencoder):
  predictions = autoencoder.predict(X_valid)

  # X_test データと再現データの mse(平均２乗誤差)を計算する
  mse = np.mean(np.power(X_valid - predictions, 2), axis=1)

  error_df = pd.DataFrame(mse).join(y_valid)
  error_df = error_df.rename(columns = {0:"reconstruction_error", "isfraut":"true_class"})
  #error_df = error_df.rename(columns = {"reconstruction_error":0, "true_class":"isfraut"})
  error_df = error_df.rename(columns = {"isfraud":"true_class"})
  
  print("true_class = 1.0")
  print(error_df[error_df.true_class == 1.0].describe())
  print(" ")
  print("true_class = 0.0")
  print(error_df[error_df.true_class == 0.0].describe())
  print(" ")
  
  fpr, tpr, thresholds = roc_curve(error_df.true_class, error_df.reconstruction_error)
  roc_auc = auc(fpr, tpr)

  plt.title('ROC curve')
  plt.plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
  plt.legend(loc='lower right')
  plt.plot([0,1],[0,1],'r--')
  plt.xlim([-0.001, 1])
  plt.ylim([0, 1.001])
  plt.ylabel('True Positive Rate')
  plt.xlabel('False Positive Rate')
  plt.show();


  # get data(precision, recall, th)
  precision, recall, th = precision_recall_curve(error_df.true_class, error_df.reconstruction_error)

  # Precision & Recall  vs  Reconstruction error
  plt.plot(th, precision[1:], 'b', label='Precision curve', color='r')
  plt.plot(th, recall[1:], 'b', label='Recall curve', color='b')
  plt.title('Precision and Recall  vs  mse')
  plt.xlabel('mse')
  plt.ylabel('Precision and Recall')
  #plt.xlim(0, 250)
  plt.xlim([0, 1.0]) #I'm sorry that I should have put this.
  plt.legend(loc=1)
  plt.show()
  
  print(" ")
  # Reconstruction error for different classes
  thresholds = [0.1,0.15, 0.2,0.25, 0.3]
  
  for threshold in thresholds:
      
    groups = error_df.groupby('true_class')
    fig, ax = plt.subplots()

    for name, group in groups:
        ax.plot(group.index, group.reconstruction_error, marker='o', ms=0.1, linestyle='',
                label= "Fraud" if name == 1 else "Normal")
    ax.hlines(threshold, ax.get_xlim()[0], ax.get_xlim()[1], colors="g", zorder=100, label='Threshold')
    ax.legend()
    ax.set_ylim(0, 1.0)  
    #ax2.set_ylim(0, .12)  
    plt.title("mse for different classes")
    plt.ylabel("mse")
    plt.xlabel("Data point index")
    plt.show();

    # Confusion matrix
    y_pred = [1 if e > threshold else 0 for e in error_df.reconstruction_error.values]
    conf_matrix = confusion_matrix(error_df.true_class, y_pred)

    plt.figure(figsize=(5, 5))
    LABELS = ["Normal", "Fraud"]
    sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d", square = True);
    plt.title("Confusion matrix")
    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.show()

In [ ]:
# baseline_model

encoder = Dense(encoding_dim, activation="relu", input_shape=(input_dim,), use_bias = True) 
decoder = Dense(input_dim, activation="relu", use_bias = True)

autoencoder_0 = Sequential()
autoencoder_0.add(encoder)
autoencoder_0.add(decoder)

autoencoder_0.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')
autoencoder_0.summary()

autoencoder_trained_0 = train_AE(autoencoder_0)

result_visualization(autoencoder_trained_0)

In [ ]:
#Autoencoder Optimization
class DenseTied(Layer):
    def __init__(self, units,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 tied_to=None,
                 **kwargs):
        self.tied_to = tied_to
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super().__init__(**kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.input_spec = InputSpec(min_ndim=2)
        self.supports_masking = True
                
    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]

        if self.tied_to is not None:
            self.kernel = K.transpose(self.tied_to.kernel)
            self._non_trainable_weights.append(self.kernel)
        else:
            self.kernel = self.add_weight(shape=(input_dim, self.units),
                                          initializer=self.kernel_initializer,
                                          name='kernel',
                                          regularizer=self.kernel_regularizer,
                                          constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)

    def call(self, inputs):
        output = K.dot(inputs, self.kernel)
        if self.use_bias:
            output = K.bias_add(output, self.bias, data_format='channels_last')
        if self.activation is not None:
            output = self.activation(output)
        return output

In [ ]:
encoder = Dense(encoding_dim, activation="relu", input_shape=(input_dim,), use_bias = True) 
decoder = DenseTied(input_dim, activation="relu", tied_to=encoder, use_bias = True)
autoencoder_1 = Sequential()
autoencoder_1.add(encoder)
autoencoder_1.add(decoder)
autoencoder_1.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')
autoencoder_1.summary()

autoencoder_trained_1 = train_AE(autoencoder_1)

result_visualization(autoencoder_trained_1)

In [ ]:
class WeightsOrthogonalityConstraint (Constraint):
    def __init__(self, encoding_dim, weightage = 1.0, axis = 0):
        self.encoding_dim = encoding_dim
        self.weightage = weightage
        self.axis = axis
        
    def weights_orthogonality(self, w):
        if(self.axis==1):
            w = K.transpose(w)
        if(self.encoding_dim > 1):
            m = K.dot(K.transpose(w), w) - K.eye(self.encoding_dim)
            return self.weightage * K.sqrt(K.sum(K.square(m)))
        else:
            m = K.sum(w ** 2) - 1.
            return m

    def __call__(self, w):
        return self.weights_orthogonality(w)

In [ ]:
encoder = Dense(encoding_dim, activation="relu", input_shape=(input_dim,), use_bias=True, kernel_regularizer=WeightsOrthogonalityConstraint(encoding_dim, weightage=1., axis=0)) 
decoder = Dense(input_dim, activation="relu", use_bias = True, kernel_regularizer=WeightsOrthogonalityConstraint(encoding_dim, weightage=1., axis=1))

autoencoder_2 = Sequential()
autoencoder_2.add(encoder)
autoencoder_2.add(decoder)

autoencoder_2.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')
autoencoder_2.summary()

autoencoder_trained_2 = train_AE(autoencoder_2)

result_visualization(autoencoder_trained_2)

In [ ]:
class UncorrelatedFeaturesConstraint (Constraint):
    
    def __init__(self, encoding_dim, weightage = 1.0):
        self.encoding_dim = encoding_dim
        self.weightage = weightage
    
    def get_covariance(self, x):
        x_centered_list = []

        for i in range(self.encoding_dim):
            x_centered_list.append(x[:, i] - K.mean(x[:, i]))
        
        x_centered = tf.stack(x_centered_list)
        covariance = K.dot(x_centered, K.transpose(x_centered)) / tf.cast(x_centered.get_shape()[0], tf.float32)
        
        return covariance
            
    # Constraint penalty
    def uncorrelated_feature(self, x):
        if(self.encoding_dim <= 1):
            return 0.0
        else:
            output = K.sum(K.square(self.covariance - K.dot(self.covariance, K.eye(self.encoding_dim))))
            return output

    def __call__(self, x):
        self.covariance = self.get_covariance(x)
        return self.weightage * self.uncorrelated_feature(x)

In [ ]:
encoder = Dense(encoding_dim, activation="relu", input_shape=(input_dim,), use_bias = True, activity_regularizer=UncorrelatedFeaturesConstraint(encoding_dim, weightage = 1.)) 
decoder = Dense(input_dim, activation="relu", use_bias = True)

autoencoder_3 = Sequential()
autoencoder_3.add(encoder)
autoencoder_3.add(decoder)

autoencoder_3.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')
autoencoder_3.summary()

autoencoder_trained_3 = train_AE(autoencoder_3)

result_visualization(autoencoder_trained_3)

In [ ]:
encoder = Dense(encoding_dim, activation="relu", input_shape=(input_dim,), use_bias = True, kernel_constraint=UnitNorm(axis=0)) 
decoder = Dense(input_dim, activation="relu", use_bias = True, kernel_constraint=UnitNorm(axis=1))
autoencoder_4 = Sequential()
autoencoder_4.add(encoder)
autoencoder_4.add(decoder)
autoencoder_4.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')
autoencoder_4.summary()

autoencoder_trained_4 = train_AE(autoencoder_4)

result_visualization(autoencoder_trained_4)

In [ ]:
encoder = Dense(encoding_dim, activation="relu", input_shape=(input_dim,), use_bias = True, kernel_regularizer=WeightsOrthogonalityConstraint(encoding_dim, weightage=1., axis=0), kernel_constraint=UnitNorm(axis=0)) 
decoder = DenseTied(input_dim, activation="relu", tied_to=encoder, use_bias = False)
autoencoder_5 = Sequential()
autoencoder_5.add(encoder)
autoencoder_5.add(decoder)
autoencoder_5.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                      optimizer='adam')
autoencoder_5.summary()

autoencoder_trained_5 = train_AE(autoencoder_5)

result_visualization(autoencoder_trained_5)

Well, I feel that it is really difficult to set proper threshold.
The Auc and ROC carve are not so bad, but I can't take advantage of them. So, plz give me advice! 

In [ ]:
autoencoder_trained_0.save("model_0.h5")

In [ ]:
autoencoder_trained_4.save("model_4.h5")